<a href="https://colab.research.google.com/github/Xixaus/Nastroje/blob/main/CD_extrakce%2Bvyhlazen%C3%AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Pro spuštění skriptu je nutné se přihlásit do Google účtu.
2. Spusťte 1. buňku tlačítkem ▶ v levém horním rohu nebo kliknutím do buňky a stistnutím klávesové zkratky ctrl+enter aby se načetly knihovny a vytvořili složky.
3. Do složky Samples vložte soubory s vzorky a do složky blanks vložte soubory s blanky

    !!! Je nutné aby pH vzoku bylo v názvu v souboru v následujícím formátu: ... _ 7_05_pufr.txt, kdy tímto způsobem bude končit název souboru !!!

3. Spusťte 2. buňku pro vygenerování grafu a souboru
    
    !!! pokud !!!
4. Soubor data.txt vložte do Bestselu, pokud Vám bude hlásit chybu, zkopírujte všechny data ze souboru a vložte do okna v Bestselu

In [1]:
#@title Volby { run: "auto", vertical-output: true }

choice = "Elipticita" # @param ["Elipticita", "Absorbance"]

In [2]:
#@title Knihovny a vytvoření/smazání složek { run: "auto", vertical-output: true }

#načtení knihoven
import os
import shutil
import pandas as pd
from scipy.signal import savgol_filter
import plotly.graph_objects as go

#vytvoření potřebných složek pro data
if os.path.exists("/content/samples"):
    shutil.rmtree("/content/samples")
os.makedirs("/content/samples", exist_ok=True)

if os.path.exists("/content/blanks"):
    shutil.rmtree("/content/blanks")
os.makedirs("/content/blanks", exist_ok=True)

#smazání zbytečné složky
if os.path.exists("/content/sample_data"):
    shutil.rmtree("/content/sample_data")

In [ ]:
#@title Výpočty { run: "auto", vertical-output: true }

#funkce pro extrakci dat ze souborů

def extract_data(folder_path, col):
    file_list = os.listdir(folder_path)
    df = pd.DataFrame() #vytvoření prázného dataframe

    for filename in file_list: #iterace každého souboru ve složce
        if filename.endswith(".txt"): #podmínka pro extrakci pouze z .txt souborů
            number = (filename.split("_")[-3].split(".")[0]) + "." + filename.split("_")[-2].split(".")[0] #pojmenování spektra, konkrétně pH, které je na konci ve formátu ..._7_05_pufr.txt
            data = pd.read_csv(f"{folder_path}/{filename}", delimiter='\t', usecols=[0,col], header=None) #extrakce vlnových délek

            start = data[data[0] == "XYDATA"].index[0] + 1

            try:
                end = data[data == "##### Extended Information"].dropna(how='all').index.item()
            except:
                end = None

            axis = data.iloc[start:end, 0].values.flatten()
            spectrum = data.iloc[start:end, [col]]
            spectrum = spectrum.rename(columns={col: float(number)})#přejmenování spektra podle promněnné number
            spectrum.index = axis

            df = pd.concat([df, spectrum], axis=1) #spojení dataframe se spectrem

    df = df.sort_index(axis=1) #seřazení sepekter podle názvu (number)
    df = df.applymap(lambda x: float(x.replace(',', '.')))
    df.index = df.index.to_series().apply(lambda x: float(x.replace(',', '.')))
    return(df)

directory = {"Elipticita": 1, "Absorbance": 2,"Flurescence": 3}

col = directory[choice]

samples = extract_data("/content/samples", col) #extrakce naměřených spekter
blanks = extract_data("/content/blanks", col) #extrakce blanků

spectra = samples - blanks
spectra_SG = samples - blanks #odečtení blanků od spekter


for column in spectra_SG.columns:
    spectra_SG[column] = savgol_filter(spectra_SG[column], window_length=7, polyorder=2, deriv=0) #vyhlazení pomocí savitzkého-golayových filtrů

fig1 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig1.add_trace(go.Scatter(y=spectra[column], x=spectra.index, mode='lines', name=str(column)))

fig1.update_layout(xaxis_title="λ [nm]",
                   yaxis_title=choice)
fig1.update_layout(title="Spektra")
fig1.show()


display(spectra)


spectra.to_csv('spectra.txt', sep='\t') #uložení spekter do .txt souboru
spectra_SG.to_csv('spectra_vyhlazena.txt', sep='\t') #uložení spekter do .txt souboru

,1.05,2.05,3.05,4.05,5.05,6.05,7.05
300.0,0.075216,0.105820,-0.040788,0.046486,0.056876,-0.004750,0.029828
299.0,-0.017966,-0.114045,-0.066426,-0.033893,-0.028604,-0.072338,-0.039193
298.0,-0.106846,0.002562,0.018879,-0.060094,-0.032146,-0.060575,-0.109941
297.0,-0.060000,0.008949,-0.016492,-0.054280,-0.017353,-0.103543,-0.106545
296.0,0.130671,0.022926,0.038068,-0.026827,0.072927,0.047279,0.013972
...,...,...,...,...,...,...,...
204.0,1.760440,0.719620,1.117720,1.180750,0.300000,0.143620,0.408160
203.0,0.556290,0.502340,1.023750,0.011870,-0.443510,-0.120730,0.100930
202.0,1.177550,1.865007,-0.338150,0.193850,0.126600,-0.292610,0.001490
201.0,1.534270,1.816508,1.931275,1.071060,-0.263160,-0.191900,-0.332040


In [ ]:
fig2 = go.Figure() #vykreslení grafu

for column in spectra.columns:
    fig2.add_trace(go.Scatter(y=spectra_SG[column], x=spectra_SG.index, mode='lines', name=str(column)))

fig2.update_layout(xaxis_title="λ [nm]",
                   yaxis_title=choice)
fig2.update_layout(title="Vyhlazená spektra")
fig2.show()

spectra_SG.astype("float16")

display(spectra_SG)

,3.05,4.05,5.05,6.05,7.05
350.0,0.001490,-0.018576,0.046525,0.030306,0.066549
349.0,-0.001090,-0.016949,0.020200,0.019411,0.039885
348.0,-0.003541,-0.015840,0.007361,0.012779,0.023696
347.0,-0.005863,-0.015250,0.008010,0.010410,0.017980
346.0,-0.013238,-0.020698,0.038898,0.008198,0.037025
...,...,...,...,...,...
204.0,0.843064,0.347479,0.640180,0.670625,0.519938
203.0,0.778774,0.491178,0.583374,0.790315,0.896717
202.0,0.645203,0.619095,0.724165,1.055198,0.912458
201.0,0.422634,0.752415,0.903229,1.379412,0.771188
